## Import required packages:

# Loan Defaulter Prediction:

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 50)
import mysql.connector
import warnings
warnings.filterwarnings('ignore')

In [ ]:
host = 'localhost'
database = 'project'
user = 'root'
password = 'root'

In [ ]:
try:
    connection = mysql.connector.connect(
    host=host,
    database=database,
    user=user,
    password=password,
    allow_local_infile=True
    )
    if connection.is_connected():
        print("Connected to mySql database")
except Error as e:
    print(f"Error: {e}")

# Read the data:

In [ ]:
query = "select * from lending_club_loan_two"
df = pd.read_sql(query, connection)
df.head()

# Exploratory Data Analysis (EDA):

In [ ]:
# information about the columns
df.info()

### Counting variables by Data type:

In [ ]:
df.dtypes.value_counts()

In [ ]:
## Statistical Summary of Numerical Variables of the data set
df.describe()

# Observations:
- Higest loan amount is 40000
- Highest interest rate is 30.99%

# Counting missing values in dataset:¶

In [ ]:
features_with_NA = [features for features in df.columns if df[features].isnull().sum() > 1]

print("Missing values:\n")

for feature in features_with_NA:
    print(f"{feature} = {round(df[feature].isna().sum()*100/len(df),2)} % \n")

# Analysis of target variable =

- Loan Status : Current status of the the loan.

In [ ]:
# To get extact count we have used valuecounts method -
df["loan_status"].value_counts()

In [ ]:
# pie chart
# with better visualization with value percentage in each categories we have used pie chart-

df['loan_status'].value_counts().plot(kind='pie',legend=True, startangle=85, fontsize=12, figsize=(14,6),autopct='%1.2f%%',explode= (0, 0.1))

# Obsevations:
- We have data for loan status in ratio of 80 to 20.
- Dataset is highly imbalanced.
- Dataset will be baised towards fully paid since we have more cases for fully paid category than charged off.

### Checking correlations =

In [ ]:
# correalation = df.corr()
# plt.figure(figsize=(12,7))
# sns.heatmap(corr, annot=True)
# Select only numeric columns
numeric_df = df.select_dtypes(include='number')


plt.figure(figsize=(12, 7))
sns.heatmap(numeric_df.corr(), annot=True, cmap='coolwarm')
plt.show()

# Observations:
- It has only plotted for numerical variables-
- There is strong correlation between installment and loan_amount.
- mort acc is +vely correlated with total acc, the correlation is not much high.
- pub_rec_bankruptcies is strongly correlated with pub_rec.
- total_acc is also correlated with open_acc
- also there is small correlation between annul_inc and installment

#### For better understanding we have created jointplot of loan amount and installment-

In [ ]:
 sns.jointplot(x="installment", y="loan_amnt", data=df, hue='loan_status', height=10)

### Observations-
- Loan amount and installment has linear relationship.
- But Loan installment amount is different for same loan amount.
- It may be because loan installment is calculated using some kind of formula which take loan amount, time period and other   factors into consideraion.

# visualising distrubution of loan amount-

In [ ]:
# continious variable - we will use histogram-

plt.figure(figsize=(20,6))
sns.histplot(df['loan_amnt'], bins=50)

# visualising distrubution of installments 

In [ ]:
plt.figure(figsize=(20,6))
sns.histplot(df['installment'],kde=False, bins=50)

# Observations:
- loan amount and installment have Right skwied distrubution
- because it has long right tail
  (Positively skewed distrubution)

### To check how loan amount affects the target variable we have used boxplot-

In [ ]:
 #t will also detect outliers

sns.boxplot(x='loan_status', y="loan_amnt", data=df, showmeans=True, palette='PuBuGn_r', saturation=5)

In [ ]:
# checking mean value of loan approval:

df.groupby("loan_status")["loan_amnt"].describe()

### Observations:
- Line inside the box that is Median line is almost equal for both fully paid and charged off loans.
- based on these we can make assumption that loan amount is not the factor in deciding whether the person will default or     not.
- there are few outlier values

### To check how interset rate affects the target variable we have used boxplot-

In [ ]:
sns.boxplot(x='loan_status', y="int_rate", data=df, showmeans=True, palette='PuBuGn_r', saturation=5)

In [ ]:
# summary statistics by loan status:

df.groupby("loan_status")["int_rate"].describe()

### Observations:
- Charged-off loans tend to have much higher interest rates
- there are some outlier values

### Interest rate vs Loan amount:
- Interest Rate: Interest Rate on the loan
- Loan amount: The listed amount of the loan applied for by the borrower. If at some point in time, the credit department reduces the loan amount, then it will be reflected in this value.

In [ ]:
plt.figure(figsize=(15,10))
sns.scatterplot(x='int_rate',y='loan_amnt', data=df, hue='loan_status')

### Observations:
- here we can see that as the interest rate is increasing the no of charged off loans are becoming denser(hight charged off loans as the interset rate increasing)
- but this is not the case in loan amount as we can see by graph.
- It seems that loans with high intersest rate are more likely to be unpaid

## Debt to Income ratio (dti):
- A ratio calculated using the borrower’s total monthly debt payments on the total debt obligations, excluding mortgage and the requested LC loan, divided by the borrower’s self-reported monthly income.

In [ ]:
df['dti'].describe()

In [ ]:
sns.boxplot(y='loan_status', x="dti", data=df, showmeans=True, palette='PuBuGn_r', saturation=5)

In [ ]:
df.groupby('loan_status')['dti'].describe()

### Observation:
- The distribution is extremely skewed.
- Lowest dti ratio is 0
- highest dti ratio is 9999
- There are several outliers, this may be the wroung entry of data.

### Loan Grading -->> df['grade']
- Loan grading is a classification system that involves assigning a quality score to a loan based on a borrower's credit history, quality of the collateral(something pledged as security for repayment of a loan), and the likelihood of repayment of the principal and interest.
- Loan grading is part of a lending institution's loan review or credit risk system and is usually an aspect of the credit underwriting and approval processes.
- The score takes into account not only the borrower's credit score but also a combination of several indicators of credit risk from the credit report and loan application, such as the level of guarantor support, repayment history, cash flow, projected yearly expenses, etc.

In [ ]:
df['grade'].unique() # grade given to customer

##### so we have understood there are total 7 grades given to the customers.

In [ ]:
# we will plot grade variable with loan status to know how grading affects the loan-

plt.figure(figsize=(12,7)) # to change the shape of graph
sns.countplot(x='grade', hue='loan_status', data=df,  palette='PuBuGn_r')

In [ ]:
# 1st have taken out the rows which have grade as "A" and so on
# we have used groupby based on loan status
# we got one object on which we will call grade variable and count it.
# Changed it into % values.
for grade in sorted(df["grade"].unique()):
    print(f"For grade {grade}:")
    print(df[df['grade']==grade].groupby("loan_status")["grade"].count()*100/len(df[df['grade']==grade]))
    print()

### Observations:
- From these we can see as the grade increasing percentage of charged off loans increasing.
- Default rate is incresing as we go from category A onwards.

In [ ]:
df["sub_grade"].unique()

In [ ]:
sub_grade_ordered = sorted(df["sub_grade"])
# odered subgrade variable for better visualizatio

In [ ]:
plt.figure(figsize=(12,7)) 
sns.countplot(x=sub_grade_ordered, hue='loan_status', data=df, palette='cividis')

In [ ]:
for grade in sorted(df["sub_grade"].unique()):
    print(f"For sub grade {grade}:")
    print(df[df['sub_grade']==grade].groupby("loan_status")["sub_grade"].count()*100/len(df[df['sub_grade']==grade]))
    print()

## Observations:
- E, F and G subgrades has maximum loan defalters.

### we will check how verification status is affecting the loan status-

In [ ]:
plt.figure(figsize=(12,7))
sns.countplot(x='verification_status', hue='loan_status', data=df, palette='cividis')

In [ ]:
for status in df['verification_status'].unique():
    print(f"For {status}:")
    print(df[df["verification_status"]==status].groupby("loan_status")["verification_status"].count()*100/len(df[df["verification_status"]==status]))
    print()

### Observations:
- Here we can see a trend that more verified people defaulted than the not verified
- distrubution for 'not verified', 'verified' and 'scource verified' is almost similar.
- that means current company varification process is not efficient as it should be.

### application_type:

In [ ]:
df['application_type'].unique()

In [ ]:
df['application_type'].value_counts()

### title and purpose:

In [ ]:
df['title'].nunique()

In [ ]:
df['purpose'].nunique()

In [ ]:
df['purpose'].value_counts()

In [ ]:
df.groupby('purpose')['loan_status'].value_counts(normalize=True).loc[:,'Charged Off'].sort_values()

# Observation-
- as we have seen purpose and title both have same kind of information.
- both columns title and purpose refers same information.
- but title has more unique values than purpose.
- 12% of completed loans for weddings have charged-off, but around 30% of completed small business loans have charged-off.

In [ ]:
df.drop("title", axis=1, inplace=True)

### emp_title:

In [ ]:
df["emp_title"].value_counts()

In [ ]:
df["emp_title"].nunique()

In [ ]:
df.drop("emp_title", axis=1, inplace=True)

### emp_length:

In [ ]:
df["emp_length"].unique()

In [ ]:
#creating an ordered list of employes
emp_order = ['< 1 year','1 year','2 years','3 years','4 years','5 years','6 years','7 years','8 years','9 years','10+ years']

In [ ]:
plt.figure(figsize=(12,7))
sns.countplot(x=df['emp_length'], order=emp_order, hue='loan_status', data=df, palette='PuBuGn_r')

In [ ]:
for value in emp_order:
    print(f"For grade {value}:")
    print(df[df['emp_length']==value].groupby("loan_status")["emp_length"].count()*100/len(df[df['emp_length']==value]))
    print()

### Observations:
- The majority of people who took loan has 10 + years of experience
- But the Distribution is somewhat uniform across all the employee length.
- Here this is not a very deciding factor whether someone will charged off or not.

In [ ]:
df.drop("emp_length", axis=1, inplace=True)

### mort_acc

In [ ]:
df["mort_acc"].count()

In [ ]:
#mortage accounts count
df["mort_acc"].value_counts()

### pub_rec:

In [ ]:
df['pub_rec'].value_counts()

In [ ]:
# we have considered 0 as a as good customer
# rest all as a bad customer.
df['pub_rec']=df['pub_rec'].apply(lambda x : 0 if x==0 else 1 )

df['pub_rec_bankruptcies']=df['pub_rec_bankruptcies'].apply(lambda x : 0 if x==0 else 1 )

In [ ]:
df['pub_rec'].value_counts()

In [ ]:
df['pub_rec_bankruptcies'].value_counts()

### home_ownership:

In [ ]:
df['home_ownership'].value_counts()

In [ ]:
# here NONE and ANY categories have very less count of values so we have added that values into the OTHER category-

df['home_ownership'] = df['home_ownership'].replace(['NONE','ANY'], 'OTHER')

In [ ]:
df['home_ownership'].value_counts()

In [ ]:
df['home_ownership'].value_counts().plot(kind='pie',legend=True, startangle=15, fontsize=12, figsize=(14,6),autopct='%1.2f%%')

In [ ]:
df.isnull().sum()

### Handling Categorical Variables:

In [ ]:
df.select_dtypes("object").columns

### loan_status:

In [ ]:
df['loan_status'].unique()

In [ ]:
df['loan_status']=df['loan_status'].map({'Fully Paid':1,'Charged Off':0})

### initial_list_status:

In [ ]:
df['initial_list_status'].unique()

In [ ]:
df['initial_list_status']=df['initial_list_status'].map({'w':0,'f':1})

In [ ]:
df['initial_list_status'].unique()

### term:

In [ ]:
df['term'].unique()

In [ ]:
for value in df['term'].unique():
    print(f"For {value}:")
    print(df[df['term']==value].groupby("loan_status")["term"].count()*100/len(df[df['term']==value]))
    print()

### address:

In [ ]:
df['address']

In [ ]:
df['address'].nunique()

In [ ]:
df['postal_code'] = df['address'].apply(lambda x: x[-5:])

In [ ]:
# as we have seen address column has 392898 unique values in it so it will create
# it will create a problem of curse of dimensionality.
# so we have extracted only the zipcode from the address and used it-

df.drop("address", axis=1, inplace=True)

In [ ]:
df['postal_code'].nunique()

In [ ]:
df['postal_code'].unique()

### earliest_cr_line:

In [ ]:
df['earliest_cr_line']

In [ ]:
#earliesr cr line extracting the year only
df["earliest_cr_line"]=df["earliest_cr_line"].apply(lambda x:int(x[-4:]))

In [ ]:
df['earliest_cr_line'].unique()

# issue_date:
- The month which the loan was funded.
- we're only using variables available to investors before the loan was funded.
- issue_d will not be included in the final model.

In [ ]:
#dropping issue date as the loan defaulter will not be dependent on issue date.
df.drop("issue_d",axis=1,inplace=True)

### grade:

In [ ]:
df['grade'].unique()

In [ ]:
sorted_grades = sorted(df['grade'].unique())

In [ ]:
ordinals = [x for x in range(1,df['grade'].nunique()+1)]

In [ ]:
grade_map = dict(zip(sorted_grades,ordinals))
grade_map

In [ ]:
df['grade'] = df['grade'].map(grade_map)
df['grade'].unique()

In [ ]:
# As grade and subgrade is expressing similar kind of information so dropping subgrade colomn-
df.drop('sub_grade', inplace=True, axis=1)

### Outliers removal:

In [ ]:
df.shape

In [ ]:
max_threshold = df['loan_amnt'].quantile(0.99)
min_threshold = df['loan_amnt'].quantile(0.01)
df = df[(df['loan_amnt']<max_threshold) & (df['loan_amnt']>min_threshold)]
df.shape

In [ ]:
max_threshold = df['int_rate'].quantile(0.99)
min_threshold = df['int_rate'].quantile(0.01)
df = df[(df['int_rate']<max_threshold) & (df['int_rate']>min_threshold)]
df.shape

In [ ]:
max_threshold = df['dti'].quantile(0.99)
min_threshold = df['dti'].quantile(0.01)
df = df[(df['dti']<max_threshold) & (df['dti']>min_threshold)]
df.shape

In [ ]:
max_threshold = df['installment'].quantile(0.99)
min_threshold = df['installment'].quantile(0.01)
df = df[(df['installment']<max_threshold) & (df['installment']>min_threshold)]
df.shape

In [ ]:
# df.to_csv("visualization.csv")

### Labal encoding:

In [ ]:
df.select_dtypes(['object']).columns

In [ ]:
df['postal_code'].unique()

In [ ]:
df['home_ownership'].unique()

In [ ]:
from sklearn.preprocessing import LabelEncoder
def label_encoding(data,var):
    encoder = LabelEncoder()
    data[var] = encoder.fit_transform(data[var])

In [ ]:
cols = df.select_dtypes(['object']).columns

for var in cols:
    label_encoding(df,var)

In [ ]:
df['postal_code'].unique()

In [ ]:
df['home_ownership'].unique()

### correlations:

In [ ]:
plt.figure(figsize=(25,15))
corelation = df.corr()
sns.heatmap(data=corelation, annot=True)

In [ ]:
# These columns dropped due to above correlations-

df.drop(['initial_list_status','pub_rec_bankruptcies','application_type','open_acc'],axis =1,inplace =True)

In [ ]:
df

### Creating dependent and independent variables:

In [ ]:
#df.to_csv('Complete_dataset.csv', index=False)

In [ ]:
x=df.drop("loan_status",axis=1)
y=df["loan_status"]

In [ ]:
x.shape, y.shape

## SMOTE (Synthetic Minority Over-sampling Technique):
- SMOTE is a valuable tool in machine learning when dealing with imbalanced datasets, helping to create more robust and fair models.

In [ ]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(sampling_strategy=0.3)
x_res, y_res = smote.fit_resample(x,y)

x_res.shape, y_res.shape

In [ ]:
x_res

## Feature selection method:

In [ ]:
from sklearn.feature_selection import mutual_info_classif

# determining the mutual information
mutual_info= mutual_info_classif(x_res,y_res)
mutual_info

In [ ]:
mutual_info = pd.Series(mutual_info)
mutual_info.index = x_res.columns
mutual_info.sort_values(ascending=False)

# here info.sort values return numpy value and not an array so convertin
# X_train and Y_train into dataframes to get x_train.columns function

In [ ]:
# poting the graph as per values of mutual info per feature
mutual_info.sort_values(ascending= False).plot.bar(figsize=(20,8))

In [ ]:
#x_res.drop(['earliest_cr_line','pub_rec', 'verification_status','annual_inc','purpose'],axis=1,inplace= True)

### Train Test Split:

In [ ]:
from sklearn.model_selection import train_test_split

X_train, x_test, y_train, y_test = train_test_split(x_res, y_res, test_size=0.2, random_state=123456)

In [ ]:
plt.figure(figsize=(25,15))

sns.heatmap(data=df.corr(), annot=True)

## Creating the Model:

In [ ]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
xg = XGBClassifier(use_label_encoder=False)
xg.fit(X_train,y_train)
y_pred = xg.predict(x_test)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(classification_report(y_test, y_pred))
print(f"Accuracy Score: {accuracy_score(y_test, y_pred) * 100:.2f}%")

In [ ]:
final_model = xg

## Saving Model : 

In [ ]:
# import pickle
# with open('./bestmodelxgboost.pkl', 'wb') as file:
#     pickle.dump(final_model, file)

In [ ]:
# import joblib

# joblib.dump(final_model,'bestmodexgboost.joblib')